## Example of BOFM as feature extractor stage.

In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import modelos
from sklearn.model_selection import StratifiedKFold    
from tensorflow.keras.utils import to_categorical
from sklearn import metrics
from BoFM.BoFM_extraccion import BoF
import scipy.io
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import AveragePooling1D,Conv2D, Layer, Dense, Conv1D, Input, Concatenate, BatchNormalization, Activation, GlobalAveragePooling1D, Flatten, MaxPool1D, LSTM
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from Gates.Gates import GMU, mGMU

reduce = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)
"""Save the model after every epoch. save_best_only saves only the best"""
#best_model = ModelCheckpoint(filepath= 'inception_gate.hdf5', monitor='loss', save_best_only=True)
#callbacks = [reduce, best_model]
callbacks = [reduce]

## Loading dataset

In [2]:
window_size_=40
data = scipy.io.loadmat('UAH_timeSeries/UAH')
data_target = data.get('data_target')
clase  = data.get('clases')
acc  = data.get('acc')
gps  = data.get('gps')
proc_vehicle  = data.get('proc_vehicle')
proc_openstreetmap  = data.get('proc_openstreetmap')
proc_lane  = data.get('proc_lane')
num_clases = np.unique(clase).shape[0]

axes=3

## kfold cross validation

In [3]:
## 10 folds
folds_=10

## indexes for every fold
cv = StratifiedKFold(n_splits=folds_, shuffle=True)
train_index_= []
test_index_=[]
for train_index, test_index in cv.split(acc,clase):  
    train_index_.append(train_index)
    test_index_.append(test_index) 

## Classifier

In [4]:
recall = []
precision =  []
facc = []
f1score = []
auc = []

for particion in range (10):                        
    train_index = train_index_[particion]
    test_index = test_index_[particion]
    y_train, y_test = data_target[train_index], data_target[test_index]
    
    #Getting the channels to be used for all inputs
    acc1_train = acc[train_index][:,:,[1,2,3]]
    acc2_train = acc[train_index][:,:,[4,5,6]]
    acc3_train = acc[train_index][:,:,[7,8,9]]
    acc1_test, acc2_test, acc3_test = acc[test_index][:,:,[1,2,3]], acc[test_index][:,:,[4,5,6]], acc[test_index][:,:,[7,8,9]]
    gps1_train, gps2_train = gps[train_index][:,:,[0,6,7]], gps[train_index][:,:,[1,2,3]]
    gps1_test, gps2_test = gps[test_index][:,:,[0,6,7]], gps[test_index][:,:,[1,2,3]]        
    proc_lane_train = proc_lane[train_index][:,:,[0,1,2]]
    proc_lane_test  = proc_lane[test_index][:,:,[0,1,2]]        
    proc_vehicle_train = proc_vehicle[train_index][:,:,[0,1,2]]
    proc_vehicle_test  = proc_vehicle[test_index][:,:,[0,1,2]]        
    proc_openstreetmap_train = proc_openstreetmap[train_index][:,:,[0,3,4]]
    proc_openstreetmap_test = proc_openstreetmap[test_index][:,:,[0,3,4]]  
    
    #Concatenate data.
    data = np.concatenate((
        acc1_train,
        acc2_train,
        acc3_train,
        gps1_train, 
        gps2_train,
        proc_lane_train,
        proc_vehicle_train,
        proc_openstreetmap_train    
    ), axis = 2)
    
    data_test = np.concatenate((
        acc1_test,
        acc2_test,
        acc3_test,
        gps1_test, 
        gps2_test,
        proc_lane_test,
        proc_vehicle_test,
        proc_openstreetmap_test    
    ), axis = 2)
    
    ##delete model
    try:
        del model           
    except(NameError):
        print("Out of memory")
    #
    #New Model
    #
    #InputSize
    input_0 = Input (shape=(55, 3))
    input_1 = Input (shape=(55, 3))
    input_2 = Input (shape=(55, 3))
    input_3 = Input (shape=(55, 3))
    input_4 = Input (shape=(55, 3))
    input_5 = Input (shape=(55, 3))
    input_6 = Input (shape=(55, 3))
    input_7 = Input (shape=(55, 3))    
    
    union, z =  mGMU(8, name="union")([input_0, input_1, input_2, input_3, input_4, input_5, input_6, input_7])
    #union_output = GlobalAveragePooling1D()(union)     
    #output  = Dense(num_clases, activation='softmax')(union_output)
    output_  = Flatten()(union)
    output   = Dense(num_clases, activation='softmax')(output_)
    model    = Model([input_0, input_1, input_2, input_3, input_4, input_5, input_6, input_7], output)
    
    #
    #Trainning BoFM. just first stage (Histograms)    
    #
    bof=BoF(z=.75, njobs=8, tipo=2)
    bof.fit(data, np.argmax(y_train, axis=1))    
    
    extraccion = bof.getHtrain()            
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])                       
    #logging.info(model.summary())    
    model.fit([extraccion[:,:,:3], extraccion[:,:,3:6], extraccion[:,:,6:9], extraccion[:,:,9:12], extraccion[:,:,12:15], extraccion[:,:,15:18], extraccion[:,:,18:21], extraccion[:,:,21:24]], y_train, epochs=250, verbose=False, callbacks=callbacks)     
    test_input = bof.transform(data_test)           
    pred=model.predict([test_input[:,:,:3], test_input[:,:,3:6], test_input[:,:,6:9], test_input[:,:,9:12], test_input[:,:,12:15], test_input[:,:,15:18], test_input[:,:,18:21], test_input[:,:,21:24]])    
    pred=np.argmax(pred, axis=1)            
    pred= to_categorical(pred, num_clases)    
    
    #Metrics
    recall_ = dict()
    precision_ =  dict()
    acc_ = dict()
    f1score_ = dict()
    fpr=dict()
    tpr=dict()
    roc_auc=dict()
    for i in range(num_clases):        
        recall_[i]    = metrics.recall_score(y_test[:,i], pred[:,i])
        precision_[i] = metrics.average_precision_score(y_test[:,i], pred[:,i])
        acc_[i]       = metrics.accuracy_score(y_test[:,i], pred[:,i])
        f1score_[i]   = metrics.f1_score(y_test[:,i], pred[:,i])
        fpr[i], tpr[i], _ = metrics.roc_curve(y_test[:,i], pred[:,i])
        roc_auc[i]=metrics.auc(fpr[i], tpr[i])
    auc.append(roc_auc)
    recall.append(recall_)
    precision.append(precision_)
    facc.append(acc_)
    f1score.append(f1score_)
    

Out of memory
4/4 [==============================] - 0s 4ms/step


## Metrics

In [5]:
f1score_promedio = np.zeros((num_clases,1))
recall_promedio = np.zeros((num_clases,1))
acc_promedio = np.zeros((num_clases,1))
precision_promedio = np.zeros((num_clases,1))
auc_promedio = np.zeros((num_clases,1))

for i in range(10):
    for j in range(num_clases):
        f1score_promedio[j] += f1score[i][j]
        recall_promedio[j] += recall[i][j]
        acc_promedio[j] += facc[i][j]
        precision_promedio[j] += precision[i][j]
        auc_promedio[j] += auc[i][j]

precision_promedio = precision_promedio/10
recall_promedio = recall_promedio/10
acc_promedio = acc_promedio/10
f1score_promedio = f1score_promedio/10
auc_promedio = auc_promedio/10

In [6]:
print("Precision: "+ str(precision_promedio))
print("Recall: "+ str(recall_promedio))
print("Accuracy: "+ str(acc_promedio))
print("F1-Score: "+ str(f1score_promedio))
print("AUC: "+ str(auc_promedio))

Precision: [[0.61419641]
 [0.75114609]
 [0.72760748]
 [0.78867754]
 [0.4594801 ]]
Recall: [[0.74934641]
 [0.84880952]
 [0.81578947]
 [0.87478632]
 [0.62315789]]
Accuracy: [[0.92350386]
 [0.92083333]
 [0.94155405]
 [0.93709781]
 [0.86328829]]
F1-Score: [[0.7501691 ]
 [0.84296558]
 [0.83104073]
 [0.86849314]
 [0.62145884]]
AUC: [[0.85278462]
 [0.89680149]
 [0.89207922]
 [0.91556543]
 [0.76940862]]
